In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [25]:
# from ngboost import NGBRegressor
# from catboost import CatBoostRegressor
# from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
from sklearn.ensemble import *
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [3]:
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, LeaveOneOut
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder

In [4]:
from bayes_opt import BayesianOptimization
from functools import partial   

In [5]:
lo = LeaveOneOut()

In [6]:
train = pd.read_csv("iris_dataset/train.csv")
test = pd.read_csv("iris_dataset/test.csv")

In [7]:
train.head()

,id,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm)
0,0,setosa,4.4,1.4,2.9,0.2
1,1,versicolor,6.4,4.5,3.2,1.5
2,2,virginica,6.2,4.8,2.8,1.8
3,3,virginica,7.2,6.1,3.6,2.5
4,4,setosa,4.9,1.4,3.0,0.2


In [8]:
test.head()

,id,species,sepal length (cm),petal length (cm)
0,0,setosa,5.4,1.7
1,1,setosa,5.7,1.5
2,2,setosa,5.3,1.5
3,3,setosa,5.1,1.9
4,4,virginica,6.0,4.8


In [9]:
train.groupby('species')['sepal width (cm)'].mean()

species
setosa        3.346667
versicolor    2.790000
virginica     2.933333
Name: sepal width (cm), dtype: float64

In [10]:
train.groupby('species')['petal width (cm)'].mean()

species
setosa        0.200000
versicolor    1.353333
virginica     2.006667
Name: petal width (cm), dtype: float64

In [11]:
train['sep_w'] = train['species'].map({'setosa' : 3, 'versicolor' : 1, 'virginica' : 2 })
test['sep_w'] = test['species'].map({'setosa' : 3, 'versicolor' : 1, 'virginica' : 2 })

In [12]:
train['pet_w'] = train['species'].map({'setosa' : 3, 'versicolor' : 1, 'virginica' : 2 })
test['pet_w'] = test['species'].map({'setosa' : 3, 'versicolor' : 1, 'virginica' : 2 })

In [13]:
train['cos_sl'] = np.cos(train['sepal length (cm)'])
test['cos_sl'] = np.cos(test['sepal length (cm)'])

In [14]:
train['cos_pl'] = np.cos(train['petal length (cm)'])
test['cos_pl'] = np.cos(test['petal length (cm)'])

In [15]:
train['sin_sl'] = np.cos(train['sepal length (cm)'])
test['sin_sl'] = np.cos(test['sepal length (cm)'])

In [16]:
train['sin_pl'] = np.cos(train['petal length (cm)'])
test['sin_pl'] = np.cos(test['petal length (cm)'])

In [17]:
train.head()

,id,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm),sep_w,pet_w,cos_sl,cos_pl,sin_sl,sin_pl
0,0,setosa,4.4,1.4,2.9,0.2,3,3,-0.307333,0.169967,-0.307333,0.169967
1,1,versicolor,6.4,4.5,3.2,1.5,1,1,0.993185,-0.210796,0.993185,-0.210796
2,2,virginica,6.2,4.8,2.8,1.8,2,2,0.996542,0.087499,0.996542,0.087499
3,3,virginica,7.2,6.1,3.6,2.5,2,2,0.608351,0.983268,0.608351,0.983268
4,4,setosa,4.9,1.4,3.0,0.2,3,3,0.186512,0.169967,0.186512,0.169967


In [18]:
test.head()

,id,species,sepal length (cm),petal length (cm),sep_w,pet_w,cos_sl,cos_pl,sin_sl,sin_pl
0,0,setosa,5.4,1.7,3,3,0.634693,-0.128844,0.634693,-0.128844
1,1,setosa,5.7,1.5,3,3,0.834713,0.070737,0.834713,0.070737
2,2,setosa,5.3,1.5,3,3,0.554374,0.070737,0.554374,0.070737
3,3,setosa,5.1,1.9,3,3,0.377978,-0.323290,0.377978,-0.323290
4,4,virginica,6.0,4.8,2,2,0.960170,0.087499,0.960170,0.087499


In [19]:
X = train.drop(['id', 'species', 'sepal width (cm)', 'petal width (cm)'],axis=1)
y1 = train.iloc[:, 4] #sepal width (cm)
y2 = train.iloc[:, 5] #petal width (cm)

In [30]:
train.head()

,id,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm),sep_w,pet_w,cos_sl,cos_pl,sin_sl,sin_pl
0,0,setosa,4.4,1.4,2.9,0.2,3,3,-0.307333,0.169967,-0.307333,0.169967
1,1,versicolor,6.4,4.5,3.2,1.5,1,1,0.993185,-0.210796,0.993185,-0.210796
2,2,virginica,6.2,4.8,2.8,1.8,2,2,0.996542,0.087499,0.996542,0.087499
3,3,virginica,7.2,6.1,3.6,2.5,2,2,0.608351,0.983268,0.608351,0.983268
4,4,setosa,4.9,1.4,3.0,0.2,3,3,0.186512,0.169967,0.186512,0.169967


In [33]:
y1.head(5)

0    2.9
1    3.2
2    2.8
3    3.6
4    3.0
Name: sepal width (cm), dtype: float64

In [34]:
y2.head(5)

0    0.2
1    1.5
2    1.8
3    2.5
4    0.2
Name: petal width (cm), dtype: float64

In [20]:
target = test[X.columns]

In [21]:
target

,sepal length (cm),petal length (cm),sep_w,pet_w,cos_sl,cos_pl,sin_sl,sin_pl
0,5.4,1.7,3,3,0.634693,-0.128844,0.634693,-0.128844
1,5.7,1.5,3,3,0.834713,0.070737,0.834713,0.070737
2,5.3,1.5,3,3,0.554374,0.070737,0.554374,0.070737
3,5.1,1.9,3,3,0.377978,-0.323290,0.377978,-0.323290
4,6.0,4.8,2,2,0.960170,0.087499,0.960170,0.087499
...,...,...,...,...,...,...,...,...
70,5.8,4.0,1,1,0.885520,-0.653644,0.885520,-0.653644
71,4.4,1.3,3,3,-0.307333,0.267499,-0.307333,0.267499
72,6.3,5.6,2,2,0.999859,0.775566,0.999859,0.775566
73,5.1,3.0,1,1,0.377978,-0.989992,0.377978,-0.989992


In [27]:
test

,id,species,sepal length (cm),petal length (cm),sep_w,pet_w,cos_sl,cos_pl,sin_sl,sin_pl
0,0,setosa,5.4,1.7,3,3,0.634693,-0.128844,0.634693,-0.128844
1,1,setosa,5.7,1.5,3,3,0.834713,0.070737,0.834713,0.070737
2,2,setosa,5.3,1.5,3,3,0.554374,0.070737,0.554374,0.070737
3,3,setosa,5.1,1.9,3,3,0.377978,-0.323290,0.377978,-0.323290
4,4,virginica,6.0,4.8,2,2,0.960170,0.087499,0.960170,0.087499
...,...,...,...,...,...,...,...,...,...,...
70,70,versicolor,5.8,4.0,1,1,0.885520,-0.653644,0.885520,-0.653644
71,71,setosa,4.4,1.3,3,3,-0.307333,0.267499,-0.307333,0.267499
72,72,virginica,6.3,5.6,2,2,0.999859,0.775566,0.999859,0.775566
73,73,versicolor,5.1,3.0,1,1,0.377978,-0.989992,0.377978,-0.989992


In [26]:
rf_pred1 = np.zeros((target.shape[0]))
vm_list = []
for i, idx in enumerate(zip(lo.split(X, y1))):
    tr_x, tr_y = X.iloc[idx[0][0]], y1.iloc[idx[0][0]]
    val_x, val_y = X.iloc[idx[0][1]], y1.iloc[idx[0][1]]
    
    rf = RandomForestRegressor(random_state=42, n_estimators=100, criterion='mae', max_depth=4)
    
    rf.fit(tr_x, tr_y)
    
    val_pred = rf.predict(val_x)
    val_mae = mean_absolute_error(val_y, val_pred)
    vm_list.append(val_mae)
    
    fol_pred = rf.predict(target) / 75
    rf_pred1 += fol_pred
    
print(f'75 FOLD 평균 MAE = {np.mean(vm_list)}')

75 FOLD 평균 MAE = 0.2612266666666666


In [39]:
tr_x

,sepal length (cm),petal length (cm),sep_w,pet_w,cos_sl,cos_pl,sin_sl,sin_pl
0,4.4,1.4,3,3,-0.307333,0.169967,-0.307333,0.169967
1,6.4,4.5,1,1,0.993185,-0.210796,0.993185,-0.210796
2,6.2,4.8,2,2,0.996542,0.087499,0.996542,0.087499
3,7.2,6.1,2,2,0.608351,0.983268,0.608351,0.983268
4,4.9,1.4,3,3,0.186512,0.169967,0.186512,0.169967
...,...,...,...,...,...,...,...,...
69,5.7,5.0,2,2,0.834713,0.283662,0.834713,0.283662
70,6.5,4.6,1,1,0.976588,-0.112153,0.976588,-0.112153
71,5.6,3.6,1,1,0.775566,-0.896758,0.775566,-0.896758
72,6.2,4.5,1,1,0.996542,-0.210796,0.996542,-0.210796


In [28]:
lo.split(X, y1)

<generator object BaseCrossValidator.split at 0x12e75f900>